In [4]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [2]:
import ipfshttpclient
client = ipfshttpclient.connect()
df_txt = client.cat('QmbY2ySpjQC9fPDRpGuxoWWkfepNYGBZvHrnsAnhwEVfDu')

In [33]:
df = pd.read_csv('rarible_user_data.csv')

In [34]:
df = df[df.columns[1:]]

In [35]:
df.head()

,blockNumber,blockTime,buyAmount,buyToken,buyer,contract,fee,price,sellAmount,sellToken,sellTokenId,seller,txHash
0,9561834,1582755781,190000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xab3627317c43d394eb171170f818153b976d28a3,TokenSale,9.500000e+15,190000000000000000,1,0x6a5ff3ceecae9ceb96e6ac6c76b82af8b39f0eb3,1725,0xdc181cde7f10db2892605c810943d74ba2c52b01,0x006ebcc644a66d5515725ee2697c44672b6c7459ac20...
1,10884922,1600415757,80000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xab3627317c43d394eb171170f818153b976d28a3,ERC1155SaleV2,4.000000e+15,80000000000000000,1,0xd07dc4262bcdbf85190c01c996b4c06a461d2430,4444,0xe286a0e4647416362c0f417a6ffec28f495fe1b0,0x023fd314a88cf896c902d39a3b19d10620b0f663d683...
2,12307905,1619333319,80000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xab3627317c43d394eb171170f818153b976d28a3,ExchangeV1,4.000000e+15,80000000000000000,1,0xd07dc4262bcdbf85190c01c996b4c06a461d2430,493692,0xe286a0e4647416362c0f417a6ffec28f495fe1b0,0x02f98a88961d3c5c43a29eb19e0f46cfa06182f07d79...
3,10816807,1599512099,10000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xab3627317c43d394eb171170f818153b976d28a3,ERC1155SaleV2,5.000000e+14,10000000000000000,1,0xeb744cf035429204e4a12438fd499ffd69c0b52c,73,0xd1b06286f759cd9e1c140c6311ff8f2816167e65,0x07a86e161ab8ba17d910de35947237cbdb96d55fc983...
4,10884878,1600415186,90000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xab3627317c43d394eb171170f818153b976d28a3,ERC1155SaleV2,4.500000e+15,90000000000000000,1,0xd07dc4262bcdbf85190c01c996b4c06a461d2430,4753,0xe286a0e4647416362c0f417a6ffec28f495fe1b0,0x0a0bed8a2f3c1ec30f9d487931e67a78f637ba6a8c63...


In [36]:
item_lookup = df[['sellTokenId']].drop_duplicates() # Only get unique item/description pairs
item_lookup['sellTokenId'] = item_lookup.sellTokenId.astype(str) # Encode as strings for future lookup ease

In [42]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['buyer_enc'] = le.fit_transform(df['buyer'])
df['sellTokenId_enc'] = le.fit_transform(df['sellTokenId'])

In [43]:
df['buyer_enc'].head()

0    550
1    550
2    550
3    550
4    550
Name: buyer_enc, dtype: int64

In [44]:
#df['buyer'] = df.buyer.astype(int) # Convert to int for customer ID
df = df[['sellTokenId_enc', 'sellAmount', 'buyer_enc']] # Get rid of unnecessary info
df_cleaned = df.groupby(['buyer_enc', 'sellTokenId_enc']).sum().reset_index() # Group together
df_cleaned.sellAmount.loc[df_cleaned.sellAmount == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
df_purchased = df_cleaned.query('sellAmount > 0') # Only get customers where purchase totals were positive

In [45]:
df_purchased

,buyer_enc,sellTokenId_enc,sellAmount
0,0,0,1
1,0,486,1
2,0,964,1
3,0,1324,1
4,0,1336,1
...,...,...,...
22141,853,8985,1
22142,853,9287,2
22143,853,9713,1
22144,854,948,13


In [46]:
customers = list(np.sort(df_purchased.buyer_enc.unique())) # Get our unique customers
products = list(df_purchased.sellTokenId_enc.unique()) # Get our unique products that were purchased
quantity = list(df_purchased.sellAmount) # All of our purchases


In [47]:
rows = pd.Categorical(df_purchased['buyer_enc'].astype('int64'), categories = customers)


In [48]:
rows

[0, 0, 0, 0, 0, ..., 853, 853, 853, 854, 854]
Length: 22146
Categories (855, int64): [0, 1, 2, 3, ..., 851, 852, 853, 854]

In [49]:
#rows = df_purchased.buyer.astype('category', categories = customers).cat.codes 
rows = pd.Categorical(df_purchased['buyer_enc'].astype('int64'), categories = customers)
# Get the associated row indices
#cols = df_purchased.sellTokenId.astype('category', categories = products).cat.codes 
cols = pd.Categorical(df_purchased['sellTokenId_enc'].astype('int64'), categories = products)
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))


In [50]:
purchases_sparse

<855x13101 sparse matrix of type '<class 'numpy.int64'>'
	with 22146 stored elements in Compressed Sparse Row format>

In [51]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.80229177630741

In [52]:
import random


In [53]:
def make_train(ratings, pct_test = 0.2):
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [54]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)


In [55]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
    # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
                        # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [104]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

In [57]:
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]


array([ 6.52699299e-03,  2.37947254e-03,  3.97489752e-04, -1.82652587e-05,
        1.39210610e-05])

In [58]:
from sklearn import metrics

def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

In [109]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_v = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_v = predictions[0][user,:]
        pred = user_v.dot(item_v).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
        # End users iteration

        return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [110]:
float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  

(0.544, 0.689)

In [112]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs)], product_test)


(0.529, 0.726)

In [113]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [127]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[df['sellTokenId_enc'].isin(prod_codes)]

In [128]:
customers_arr[:5]


array([0, 1, 2, 3, 4])

In [154]:
get_items_purchased(1, product_train, customers_arr, products_arr, item_lookup)


,sellTokenId
1001,3638
2192,5701
3472,5678
3561,54968
3981,8896
5522,60426
7833,90229
8632,734
8674,521894
8917,13402


In [140]:
from sklearn.preprocessing import MaxAbsScaler


In [167]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, num_items = 10):
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MaxAbsScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
#    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes}) # Create a dataframe 
    return final_frame[['StockCode']] # Switch order of columns around

In [168]:
cust_ind = np.where(customers_arr == 0)[0][0] # Returns the index row of our customer id

user_vecs[cust_ind,:].dot(item_vecs)

<1x13101 sparse matrix of type '<class 'numpy.float64'>'
	with 10987 stored elements in Compressed Sparse Row format>

In [169]:
rec_items(1, product_train, user_vecs, item_vecs, customers_arr, products_arr,
                       num_items = 10)

,StockCode
0,0
